## Prepare ROMS I4DVar observation file from a ROMS output file

In [ ]:
from dataclasses import dataclass

import cftime
import numpy as np
import xarray as xr
import xesmf as xe
import matplotlib.pyplot as plt

There are no nc stations file for this time period, get from the next period for coordinates.

In [ ]:
@dataclass
class Arguments:
    start_time: str = '2018-04-01'
    end_time: str = '2018-04-07'
    input_grid_file: str = '/cluster/projects/nn9490k/ROHO800/Grid/ROHO800_grid_fix5.nc'
    input_data_files: tuple = (
        '/cluster/projects/nn9297k/ROHO160+/OutputData/s_layers_25/1_dec2017-dec2018/roho160_his_0011.nc',
        '/cluster/projects/nn9297k/ROHO160+/OutputData/s_layers_25/1_dec2017-dec2018/roho160_his_0012.nc',
    )
    stations: str = '/cluster/projects/nn9297k/ROHO160+/OutputData/s_layers_25/2_dec2018-sep2019/roho160_sta.nc'
    wc13_obs_file: str = '/cluster/home/shmiak/src/roms-applications/WC13/Data/wc13_obs.nc'
    output_obs_file: str = '/cluster/projects/nn9297k/shmiak/roho800_data/input_data/roho800_obs_sst_roho160_2018-04-01_to_2018-04-07.nc'

args = Arguments()

Load necessary nc files to datasets.

In [ ]:
time_slice = slice(args.start_time, args.end_time)
ds_data = xr.open_mfdataset(list(args.input_data_files)).sel(ocean_time=time_slice)
ds_stations = xr.open_dataset(args.stations).sel(ocean_time=time_slice)
ds_grid = xr.open_dataset(args.input_grid_file)
ds_wc13_obs = xr.open_dataset(args.wc13_obs_file)

### Interpolate (regrid) temperature from 160 m to 800 m (checkup)

Prepare variables and resample from 160 to 800 meters domain.
Description of xesmf is unclear for me, so renaming is to follow the examples
at `https://xesmf.readthedocs.io/en/latest/notebooks/Curvilinear_grid.html`.

In [ ]:
ds_grid = ds_grid.rename({"lon_rho": "lon", "lat_rho": "lat"})
ds_data = ds_data.rename({"lon_rho": "lon", "lat_rho": "lat"})

In [ ]:
regridder = xe.Regridder(ds_data, ds_grid, "bilinear", unmapped_to_nan=True)
da_sst = regridder(ds_data['temp']).isel(s_rho=-1)

Visual check

In [ ]:
da_sst = da_sst / ds_grid.mask_rho  # type: ignore ; exclude values outside the sea

In [ ]:
ds_data['temp'].isel(ocean_time=-1, s_rho=-1).plot(figsize=(14, 7))  # type: ignore

In [ ]:
da_sst.isel(ocean_time=-1).plot(figsize=(14, 7))  # type: ignore

### Check the coordinates of the observation stations

In [ ]:
st_lons, st_lats = np.zeros(ds_stations.station.shape), np.zeros(ds_stations.station.shape)
for i, station in enumerate(ds_stations.station):
    st_lons[i] = station.lon_rho.values
    st_lats[i] = station.lat_rho.values

In [ ]:
st_labels = ['VT53', 'VT70', 'VT74', 'VT69']
st_n_point = [1, 3, 4, 9]
p = ds_grid.mask_rho.plot(
    x="lon", y="lat", figsize=(14, 7), cmap='GnBu'
    )
p.axes.scatter(x=st_lons[st_n_point], y=st_lats[st_n_point], color='red')
for i, label in enumerate(st_labels):
    p.axes.annotate(label, (st_lons[st_n_point][i], st_lats[st_n_point][i]), color='red')

### Interpolate 160 m output to the data assimilation stations

In [ ]:
ds_st_coords = xr.Dataset(
    {
        "lat": (["lat"], st_lats[st_n_point], {"units": "degrees_north"}),
        "lon": (["lon"], st_lons[st_n_point], {"units": "degrees_east"}),
    }
)

In [ ]:
regridder = xe.Regridder(ds_data, ds_st_coords, "bilinear", unmapped_to_nan=True)
da_stations_temp = regridder(ds_data['temp'])

Xesmf interpolates to the grid that is a combination of station coordinates

In [ ]:
da_stations_temp

Extract the diagonal elements along the lat and lon dimensions, these are stations

In [ ]:
na_stations = da_stations_temp.values.diagonal(axis1=2, axis2=3)

Checkup

In [ ]:
na_stations.shape

In [ ]:
na_stations[0, 3, :]

In [ ]:
da_stations_temp.isel(ocean_time=0, s_rho=3).values

### Prepare data arrays for the ROMS data assimilation observation file

In [ ]:
points_per_time = len(st_n_point) * da_stations_temp.s_rho.shape[0]
time_points = da_stations_temp.ocean_time.shape[0]
print(f"There are {points_per_time} observations per time and {time_points} time points.")

obs_value

In [ ]:
na_stations[0, 0, :]

In [ ]:
na_stations[0, 1, :]

In [ ]:
na_stations.flatten()[:8]

In [ ]:
# Flattens from the last dimenstion: (x, y, z) so z -> y -> x
na_st_temp = na_stations.flatten()
na_st_temp.shape

In [ ]:
ds_st_coords

The order of values: 'VT53', 'VT70', 'VT74', 'VT69' from the bottom upwards, from the first time step

obs_type

In [ ]:
na_type = np.full_like(na_st_temp, 6, dtype=np.int32)
na_type.shape

obs_provenance

In [ ]:
na_provenance = np.full_like(na_st_temp, 1, dtype=np.int32)
na_provenance.shape

obs_time

In [ ]:
na_time = np.repeat(da_stations_temp.ocean_time.values[..., np.newaxis], points_per_time, axis=1).flatten()
na_time.shape

In [ ]:
fig, ax = plt.subplots(figsize=(14, 7))
ax.plot(na_time)

obs_depth

If positive, should be a ROMS grid level, for example, 25 is a top layer if there are 25 layers
If negative, meters, not tested

Here should be like 1 1 1 1 2 2 2 2... 25 25 25 25 1 1 1 1 ...

In [ ]:
depth_time_step = np.repeat(np.arange(1, 26, 1)[..., np.newaxis], len(st_n_point), axis=1).flatten()

In [ ]:
na_depth = np.tile(depth_time_step, time_points)
na_depth.shape

obs_Xgrid and obs_Ygrid

In [ ]:
x_idx, y_idx = np.where(mask)

In [ ]:
np_xgrid = np.tile(x_idx.astype(dtype=np.float64) + 1, time_points)
np_ygrid = np.tile(y_idx.astype(dtype=np.float64) + 1, time_points)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 7))
ax.plot(np_xgrid)

In [ ]:
np_xgrid.shape

In [ ]:
np_ygrid.shape

obs_Zgrid

In [ ]:
np_zgrid = np.full_like(np_sst, 0)
np_zgrid.shape

obs_Error

In [ ]:
# np_sst_std = da_sst_std.values.flatten(order='C')
# np_sst_std = np_sst_std[~np.isnan(np_sst_std)]
# np_sst_var = np_sst_std ** 2
# np_error = np.repeat(np_sst_var, time_points)
np_error = np.full_like(np_sst, 0.4**2)

In [ ]:
np_error.shape

survey_time

In [ ]:
np_survey_time = da_sst.time.values

In [ ]:
np_survey_time

np_nobs

In [ ]:
np_nobs = np.repeat(points_per_time, time_points)

In [ ]:
np_nobs

np_lon and np_lat

In [ ]:
np_lon = np.tile(ds_grid.lon_rho.values.flatten()[mask.flatten()==True], time_points)

In [ ]:
np_lon.shape

In [ ]:
np_lat = np.tile(ds_grid.lat_rho.values.flatten()[mask.flatten()==True], time_points)

In [ ]:
np_lat.shape

### Make a dataset

In [ ]:
ds = xr.Dataset(
    {
        "spherical": 1,
        "Nobs": ("survey", np_nobs),
        "survey_time": ("survey", np_survey_time),
        "obs_variance": ("state_variable", wc13_obs.obs_variance.data),
        "obs_value": ("datum", np_sst),
        "obs_type": ("datum", np_type),
        "obs_provenance": ("datum", np_provenance),
        "obs_time": ("datum", np_time),
        "obs_depth": ("datum", np_depth),
        "obs_Xgrid": ("datum", np_xgrid),
        "obs_Ygrid": ("datum", np_ygrid),
        "obs_Zgrid": ("datum", np_zgrid),
        "obs_error": ("datum", np_error),
        "obs_lon": ("datum", np_lon),
        "obs_lat": ("datum", np_lat),
    },
)
ds

In [ ]:
ds.to_netcdf(args.output_obs_file)